# Appendix D -- Reinforcement Learning Foundations
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/APP_D_Reinforcement_Learning.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Prerequisites:** Chapter 7 (Deep Learning with PyTorch)  

---

### Learning Objectives

- Explain the RL framework: agent, environment, state, action, reward
- Implement Q-learning from scratch on a grid world
- Understand the Bellman equation and temporal difference learning
- Build a Deep Q-Network (DQN) agent that solves CartPole-v1
- Explain experience replay and the target network trick


---

## Setup


In [ ]:
import subprocess
subprocess.run(['pip', 'install', 'gymnasium', '-q'], check=False)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import deque, namedtuple
import random
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Gymnasium: {gym.__version__}')
print(f'Device:    {DEVICE}')

RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 110


---

## D.1 -- The Reinforcement Learning Framework

RL is fundamentally different from supervised and unsupervised learning.
There is no labelled dataset. Instead:

- An **agent** takes **actions** in an **environment**
- The environment returns a new **state** and a **reward** signal
- The agent's goal is to maximise **cumulative reward** over time

```
         action
  Agent --------> Environment
    ^                  |
    |   state, reward  |
    +------------------+
```

**Key concepts:**

**Policy (π):** the agent's strategy -- given a state, which action to take.  
**Value function V(s):** expected cumulative reward from state s.  
**Q-function Q(s,a):** expected cumulative reward from state s taking action a.  
**Discount factor γ:** how much to value future rewards vs immediate ones.  
**The Bellman equation:** `Q(s,a) = r + γ * max_a' Q(s', a')`  
(the value of an action equals its immediate reward plus the discounted
best future value from the resulting state)


In [ ]:
# D.1.1 -- Q-learning from scratch on a simple grid world

class GridWorld:
    """
    4x4 grid. Agent starts at (0,0), goal at (3,3), holes at (1,1) and (2,3).
    Actions: 0=up, 1=down, 2=left, 3=right
    Rewards: +10 goal, -10 hole, -0.1 each step (encourages efficiency)
    """
    GOAL  = (3, 3)
    HOLES = {(1, 1), (2, 3)}
    SIZE  = 4

    def reset(self):
        self.pos = (0, 0)
        return self.pos

    def step(self, action):
        r, c = self.pos
        if   action == 0: r = max(r - 1, 0)
        elif action == 1: r = min(r + 1, self.SIZE - 1)
        elif action == 2: c = max(c - 1, 0)
        elif action == 3: c = min(c + 1, self.SIZE - 1)
        self.pos = (r, c)
        if self.pos == self.GOAL:       return self.pos, +10.0, True
        if self.pos in self.HOLES:      return self.pos, -10.0, True
        return self.pos, -0.1, False


# Q-learning: learn Q(s, a) for every state-action pair
n_states  = GridWorld.SIZE ** 2   # 16 states (row*SIZE + col)
n_actions = 4
Q = np.zeros((n_states, n_actions))

env     = GridWorld()
alpha   = 0.1    # learning rate
gamma   = 0.95   # discount factor
epsilon = 1.0    # exploration rate (decays over time)
eps_min = 0.05
eps_decay = 0.995

episode_rewards = []

for episode in range(1000):
    state = env.reset()
    s     = state[0] * GridWorld.SIZE + state[1]   # flatten to int
    total_reward = 0

    for _ in range(50):   # max steps per episode
        # Epsilon-greedy: explore randomly or exploit Q table
        if random.random() < epsilon:
            action = random.randint(0, n_actions - 1)
        else:
            action = np.argmax(Q[s])

        next_state, reward, done = env.step(action)
        s_next = next_state[0] * GridWorld.SIZE + next_state[1]

        # Bellman update: Q(s,a) <- Q(s,a) + alpha * (r + gamma*max Q(s') - Q(s,a))
        Q[s, action] += alpha * (
            reward + gamma * np.max(Q[s_next]) - Q[s, action]
        )
        s = s_next
        total_reward += reward
        if done:
            break

    epsilon = max(eps_min, epsilon * eps_decay)
    episode_rewards.append(total_reward)

# Plot learning curve
window = 50
smoothed = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(episode_rewards, alpha=0.3, color='#2E75B6', linewidth=0.8)
ax.plot(range(window-1, len(episode_rewards)), smoothed, '#E8722A', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Total Reward')
ax.set_title('Q-Learning on Grid World: Learning Curve\n(orange = 50-episode moving average)')
plt.tight_layout()
plt.show()

# Show learned policy
action_symbols = ['↑', '↓', '←', '→']
print('Learned policy (greedy from Q table):')
for r in range(GridWorld.SIZE):
    row = ''
    for c in range(GridWorld.SIZE):
        if   (r, c) == GridWorld.GOAL:       row += ' G '
        elif (r, c) in GridWorld.HOLES:      row += ' X '
        else:
            s   = r * GridWorld.SIZE + c
            row += f' {action_symbols[np.argmax(Q[s])]} '
    print(row)


---

## D.2 -- Deep Q-Network (DQN) for CartPole

Q-learning with a table works only for small, discrete state spaces.
**DQN** replaces the Q-table with a neural network that approximates Q(s, a)
for continuous or high-dimensional state spaces.

Two tricks make DQN stable:

**Experience replay:** store transitions (s, a, r, s') in a buffer and
sample random mini-batches. This breaks the temporal correlation between
consecutive training samples that would otherwise make training unstable.

**Target network:** maintain a separate copy of the Q-network whose weights
are updated slowly. Using the same network for both predictions and targets
creates a moving target that causes divergence.


In [ ]:
# D.2.1 -- DQN agent for CartPole-v1
# CartPole: balance a pole on a cart by pushing left or right
# State: 4 floats (cart pos, cart vel, pole angle, pole vel)
# Actions: 0=push left, 1=push right
# Episode ends when pole falls or cart moves too far
# Solved when average reward >= 475 over 100 episodes

Transition = namedtuple('Transition', ['state','action','reward','next_state','done'])

class ReplayBuffer:
    def __init__(self, capacity=10_000):
        self.buffer = deque(maxlen=capacity)
    def push(self, *args):
        self.buffer.append(Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    def __len__(self):
        return len(self.buffer)


class DQN(nn.Module):
    def __init__(self, n_obs, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_obs, 128), nn.ReLU(),
            nn.Linear(128, 128),   nn.ReLU(),
            nn.Linear(128, n_actions)
        )
    def forward(self, x):
        return self.net(x)


env_cp      = gym.make('CartPole-v1')
n_obs       = env_cp.observation_space.shape[0]   # 4
n_actions_cp = env_cp.action_space.n              # 2

policy_net = DQN(n_obs, n_actions_cp).to(DEVICE)
target_net = DQN(n_obs, n_actions_cp).to(DEVICE)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer_dqn = optim.AdamW(policy_net.parameters(), lr=1e-3)
buffer    = ReplayBuffer(10_000)

# Hyperparameters
BATCH_SIZE    = 128
GAMMA_DQN     = 0.99
EPS_START     = 1.0
EPS_END       = 0.05
EPS_DECAY     = 500     # steps to decay over
TARGET_UPDATE = 20      # update target network every N episodes
N_EPISODES_DQN = 400

episode_durations = []
steps_done = 0

def select_action(state, steps_done):
    eps = EPS_END + (EPS_START - EPS_END) * np.exp(-steps_done / EPS_DECAY)
    if random.random() < eps:
        return random.randint(0, n_actions_cp - 1)
    with torch.no_grad():
        s = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(DEVICE)
        return policy_net(s).argmax(1).item()

def optimise_model():
    if len(buffer) < BATCH_SIZE:
        return
    batch      = buffer.sample(BATCH_SIZE)
    states     = torch.tensor(np.array([t.state      for t in batch]), dtype=torch.float32).to(DEVICE)
    actions    = torch.tensor([t.action    for t in batch], dtype=torch.long).unsqueeze(1).to(DEVICE)
    rewards    = torch.tensor([t.reward    for t in batch], dtype=torch.float32).to(DEVICE)
    next_states= torch.tensor(np.array([t.next_state for t in batch]), dtype=torch.float32).to(DEVICE)
    dones      = torch.tensor([t.done      for t in batch], dtype=torch.float32).to(DEVICE)

    # Current Q values
    q_values = policy_net(states).gather(1, actions).squeeze(1)

    # Target Q values (Bellman equation)
    with torch.no_grad():
        next_q = target_net(next_states).max(1).values
        targets = rewards + GAMMA_DQN * next_q * (1 - dones)

    loss = nn.SmoothL1Loss()(q_values, targets)
    optimizer_dqn.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 100)
    optimizer_dqn.step()

print(f'Training DQN on CartPole-v1 for {N_EPISODES_DQN} episodes...')

for episode in range(N_EPISODES_DQN):
    state, _ = env_cp.reset(seed=episode)
    for t in range(500):
        action = select_action(state, steps_done)
        next_state, reward, terminated, truncated, _ = env_cp.step(action)
        done = terminated or truncated
        buffer.push(state, action, reward, next_state, float(done))
        state = next_state
        steps_done += 1
        optimise_model()
        if done:
            episode_durations.append(t + 1)
            break
    if episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

env_cp.close()

window = 50
smoothed_dqn = np.convolve(episode_durations, np.ones(window)/window, mode='valid')
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(episode_durations, alpha=0.3, color='#2E75B6', linewidth=0.8)
ax.plot(range(window-1, len(episode_durations)), smoothed_dqn, '#E8722A', linewidth=2)
ax.axhline(475, color='green', linestyle='--', linewidth=1.5, label='Solved threshold (475)')
ax.set_xlabel('Episode')
ax.set_ylabel('Duration (steps survived)')
ax.set_title('DQN on CartPole-v1\n(agent learns to balance the pole)')
ax.legend()
plt.tight_layout()
plt.show()

final_avg = np.mean(episode_durations[-100:])
solved    = 'SOLVED' if final_avg >= 475 else f'avg={final_avg:.0f} (not yet solved -- try more episodes)'
print(f'Last 100 episodes average duration: {final_avg:.0f}  --  {solved}')


---

## D.3 -- RL Summary and Where to Go Next

### Key Takeaways

- RL learns from interaction, not labelled data. The reward signal is the only supervision.
- The **Bellman equation** is the mathematical foundation: the value of an action
  equals its immediate reward plus the discounted value of the best next action.
- **Epsilon-greedy** balances exploration (try new things) and exploitation (use what you know).
- **Experience replay** breaks temporal correlations; **target networks** stabilise training.
  Both are necessary for DQN to converge.
- CartPole is solved when the agent keeps the pole balanced for 475+ steps consistently.

### Beyond DQN

- **Policy gradient methods** (REINFORCE, PPO, A3C) -- optimise the policy directly
  rather than learning a Q-function. Better for continuous action spaces.
- **Actor-Critic (A2C/A3C)** -- combines value-based and policy-based methods.
- **Proximal Policy Optimisation (PPO)** -- the most widely used RL algorithm
  in production (used by OpenAI for ChatGPT's RLHF training).
- **Stable-Baselines3** -- the standard Python RL library;
  provides PPO, SAC, TD3 with a scikit-learn-style API.

---

*End of Appendix D -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
